In [1]:
import pandas as pd


import the csv files for the US data we will be working with, and rename a couple of columns that were unnames

In [ ]:
beer_path = "resources/beers.csv"
us_beer = pd.read_csv(beer_path)
us_beer = us_beer.rename(columns = {'Unnamed: 0':'beer_id'})
us_beer.head()

In [ ]:
brew_path = "resources/breweries.csv"
us_breweries = pd.read_csv(brew_path)
us_breweries = us_breweries.rename(columns = {"Unnamed: 0": 'brewery_id'})
#us_breweries.set_index('brewery_id', inplace = True)
us_breweries.head()

Get a count of beers produced at each brewery using groupby and count then add it to the breweries dataframe.

In [ ]:
beer_count = us_beer.groupby(['brewery_id']).count()
us_breweries['beer_count'] = beer_count['beer_id']
us_breweries.head()

Cut down the original beer dataframe to just the items we need.

In [ ]:
cut_us_beer = us_beer[['beer_id', 'name', 'style', 'abv', 'brewery_id']]
cut_us_beer.set_index('beer_id', inplace = True)
cut_us_beer.head()

In [ ]:
import requests
import json
from config import gkey


get a test list of brewery names and a full list in order to find ratings for each brewery. 

In [ ]:
short_brewery = us_breweries['name'].loc[us_breweries['brewery_id']<10]
base_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
short_brewery
brewery= us_breweries['name']

run a for loop the requests for each brewery in either the test list or the full list. and add the responses to a list.

In [ ]:
rating = []
for b in brewery:
    query_url = f'{base_url}{b}&inputtype=textquery&fields=name,rating&key={gkey}'
    response = requests.get(query_url)
    response_json = response.json()
    try:
      rating.append(response_json['candidates'][-1]['rating'])
    except:
        rating.append('NaN')
        
rating  

add the new list of ratings and the country to the breweries dataframe

In [ ]:
#us_breweries['rating'] = rating
us_breweries['country'] = 'USA'

print the dataframes to csv so that we don't have to run the for loop over and over getting me closer to that google requests limit.

In [2]:
us_breweries.to_csv('output/final_us_brewery.csv')
cut_us_beer.to_csv('output/final_us_beer.csv')

NameError: name 'us_breweries' is not defined

In [3]:
#import the csv files to manipulate a bit more before sending it to sql
us_beer_df = pd.read_csv('output/final_us_beer.csv')
us_brewery_df = pd.read_csv('output/final_us_brewery.csv')

In [4]:
#make sure names match sql table
us_beer_df = us_beer_df.rename(columns={'name':'beer_name', 'abv':'ABV'})
us_brewery_df = us_brewery_df.rename(columns = {'name':'brewery_name','Country':'country'})

In [5]:
us_beer_df

,beer_id,beer_name,style,ABV,brewery_id
0,0,Pub Beer,American Pale Lager,0.050,408
1,1,Devil's Cup,American Pale Ale (APA),0.066,177
2,2,Rise of the Phoenix,American IPA,0.071,177
3,3,Sinister,American Double / Imperial IPA,0.090,177
4,4,Sex and Candy,American IPA,0.075,177
...,...,...,...,...,...
2405,2405,Belgorado,Belgian IPA,0.067,424
2406,2406,Rail Yard Ale,American Amber / Red Ale,0.052,424
2407,2407,B3K Black Lager,Schwarzbier,0.055,424
2408,2408,Silverback Pale Ale,American Pale Ale (APA),0.055,424


In [6]:
us_brewery_df

,Unnamed: 0,brewery_id,brewery_name,city,state,beer_count,rating,country
0,0,0,NorthGate Brewing,Minneapolis,MN,6,4.2,USA
1,1,1,Against the Grain Brewery,Louisville,KY,13,4.5,USA
2,2,2,Jack's Abby Craft Lagers,Framingham,MA,5,4.6,USA
3,3,3,Mike Hess Brewing Company,San Diego,CA,6,4.6,USA
4,4,4,Fort Point Beer Company,San Francisco,CA,4,3.8,USA
...,...,...,...,...,...,...,...,...
553,553,553,Covington Brewhouse,Covington,LA,1,4.6,USA
554,554,554,Dave's Brewfarm,Wilson,WI,1,4.8,USA
555,555,555,Ukiah Brewing Company,Ukiah,CA,1,4.2,USA
556,556,556,Butternuts Beer and Ale,Garrattsville,NY,4,4.6,USA


In [7]:
#import sqlalchemy
from sqlalchemy import create_engine

In [8]:
#drop a few values
us_beer_df = us_beer_df.drop(columns = ['beer_id'])
us_brewery_df = us_brewery_df.drop(columns = ['Unnamed: 0'])

In [9]:
#establish connection
connection_string = "postgres:postgres@localhost:5432/Beer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#check that tables have been created.
engine.table_names()

In [ ]:
#push beer dataframe to SQL
us_beer_df.to_sql(name = 'beer_tbl', con = engine, if_exists='append', index = False)

In [10]:
#push brewery dataframe to SQL
us_brewery_df.to_sql(name = 'brewery_tbl', con = engine, if_exists='append', index = False)